<a href="https://colab.research.google.com/github/MadhurimaPaul-87/Natural-Language-Processing/blob/main/Spam_Classifier(lemma%2C_TF_IDF%2Chigh_Accuracy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd

In [30]:
messages = pd.read_csv("/content/SMSSpamCollection", sep='\t', names=["label","message"])

In [31]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [35]:
messages.shape

(5572, 2)

Data cleaning & Preprocessing

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

cleaning the text

In [68]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
 nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [69]:
ps = PorterStemmer()
wordnet = WordNetLemmatizer()

In [70]:
corpus = []
for i in range(len(messages)):
  res = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
  res = res.lower()
  res = res.split()
  res = [wordnet.lemmatize(word) for word in res if not word in set(stopwords.words('english'))] 
  res = ' '.join(res)
  corpus.append(res)
  corpus

Creating TfidfVectorizer model

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [72]:
cv = TfidfVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [73]:
dummy= pd.get_dummies(messages['label'])
dummy

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
5567,0,1
5568,1,0
5569,1,0
5570,1,0


In [74]:
y= dummy.drop(['ham'],axis=1)
y.head()

,spam
0,0
1,0
2,1
3,0
4,0


Train test split

In [75]:
from sklearn.model_selection import train_test_split

In [76]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [77]:
len(X_train)

4457

In [78]:
len(X_test)

1115

Training model using Naive Bayes Classifier

In [79]:
from sklearn.naive_bayes import MultinomialNB

In [80]:
spam_detect_model = MultinomialNB()
spam_detect_model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [81]:
y_pred=spam_detect_model.predict(X_test)

In [82]:
from sklearn.metrics import confusion_matrix

In [83]:
cm = confusion_matrix(y_test,y_pred)
cm

array([[971,   0],
       [ 24, 120]])

In [84]:
from sklearn.metrics import accuracy_score

In [85]:
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.97847533632287